In [9]:
import csv
import math

import pandas as pd
import requests
import numpy as np

### Dataset Location
The Dataset cannot be pushed to Github. Github has decreased the allowed file size so the dataset will have to be downloaded manually.

The url is: https://data.cms.gov/provider-data/sites/default/files/resources/69a75aa9d3dc1aed6b881725cf0ddc12_1671768321/DAC_NationalDownloadableFile.csv

The Data Dictionary for this section of datasets: https://data.cms.gov/provider-data/sites/default/files/data_dictionaries/physician/DOC_Data_Dictionary.pdf

The webpage for this section of datasets: https://data.cms.gov/provider-data/topics/doctors-clinicians

In [7]:
df = pd.read_csv('DAC_NationalDownloadableFile.csv', encoding='windows-1252', dtype='string')

### (TODO) Filtering and Cleaning the Dataset
Note: Since this is for Physicians, we will need to filter out non-physicians, but keep in mind that some physicians will also have a secondary degree listed under their specialities, so the filtering needs to be a bit smarter

In [21]:
# NOTE: Optimize This
addresses = df[['adr_ln_1', 'cty', 'st', 'zip','pri_spec']]
addresses.drop_duplicates()
addresses.dropna()
addresses = addresses[addresses['adr_ln_1'].str.strip().astype(bool)]
addresses = addresses[addresses['cty'].str.strip().astype(bool)]
addresses = addresses[addresses['st'].str.strip().astype(bool)]
addresses = addresses[addresses['zip'].str.strip().astype(bool)]
addresses = addresses[addresses['pri_spec'].str.strip().astype(bool)]
print(addresses.head())
print(addresses.shape[0])

             adr_ln_1             cty  st        zip                pri_spec
0     1582 N BROAD ST        TAZEWELL  TN  378794352      NURSE PRACTITIONER
1   5632 ANNAPOLIS RD     BLADENSBURG  MD  207102213      NURSE PRACTITIONER
2    611 E CARLSON ST        CHEYENNE  WY  820094335       INTERNAL MEDICINE
3         30 RIVER ST      JIM THORPE  PA  182292313            CHIROPRACTIC
4  20501 VENTURA BLVD  WOODLAND HILLS  CA  913640847  CLINICAL SOCIAL WORKER
2472219


In [4]:
batches = math.ceil(addresses.shape[0] / 10000)
print(batches)

247


### Steps to Get Geolocations of Addresses in Batch
1. Create a loop that generates a CSV for each 247 entries 
2. Send the generated CSV to the Census GOV Geolocation API for processing
3. Recieve the response and append it to a local CSV 
4. Rinse and repeat until all addresses are converted to geolocations (lat. and long.) to use with a map

In [5]:
URL = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch'
with open('test_addresses.csv', 'rb') as input_file:
    geo = requests.post(files={'addressFile': input_file}, url=URL, data={'benchmark':'Public_AR_Current'})

In [6]:
print(geo.content)

b'"000001","8 E View Dr, Whitehouse Station, NJ, 08889","Match","Exact","8 E VIEW DR, WHITEHOUSE STATION, NJ, 08889","-74.76707085499999,40.611750001000075","627779463","R"\n'
